<a href="https://colab.research.google.com/github/scienceofdeduction/pub/blob/master/julia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%shell
if ! command -v julia 3>&1 > /dev/null
then
    wget 'https://julialang-s3.julialang.org/bin/linux/x64/1.5/julia-1.5.0-linux-x86_64.tar.gz' \
        -O /tmp/julia.tar.gz
    tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
    rm /tmp/julia.tar.gz
    julia -e 'using Pkg; Pkg.add("IJulia"); Pkg.precompile();'
fi

In [ ]:
%%sh
which julia

![julia colab][1]

[1]: https://user-images.githubusercontent.com/19316036/90217362-0079f380-ddb6-11ea-84d4-5059a2bea538.png

In [23]:
using Pkg

SyntaxError: ignored

In [ ]:
%%sh
julia -e 'using Pkg; Pkg.status();'

In [ ]:
%%sh 
julia -e 'using Pkg; Pkg.add("Flux");'

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
install.packages("JuliaConnectoR")

In [ ]:
%%R
require(JuliaConnectoR)

In [ ]:
%%R
flux <- juliaImport("Flux")

In [ ]:
%%R

m <- Flux$Chain(
        Flux$Dense(4L, 12L, Flux$relu), 
        Flux$Dense(12L, 3L, Flux$softmax)
)

In [ ]:
%%R

mlp <- juliaEval('
  using Flux
  function mlp(x, y)
    opt = ADAM(.01)
    m = Chain(Dense(4, 12, relu), Dense(12, 3, softmax))
    loss(x, y) = crossentropy(m(x), y)
    # @epochs 10 train!(loss, params(m), [(x, y)], opt)

  end 
')